In [1]:
import skimage
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
import time

## Carrega a base e divide em treino e teste

In [2]:
inicioGeral = time.time()
bases_prontas_path = os.path.join("D:\\","FIA","TCC","BASES","")
print(bases_prontas_path)

D:\FIA\TCC\BASES\


In [3]:
df = pd.read_csv(bases_prontas_path+'mask_dataset_vgg16_preprocess_input_224_224_3_feature_extracted.csv')
X, y = df.drop(['im_path', 'class'], axis=1), df['class'].values

label_transformer = preprocessing.LabelEncoder()
label_transformer.fit(y)
y = label_transformer.transform(y)

#print(list(y))

trainX , testX, trainY, testY  = train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
dataframe_metricas_modelos = pd.DataFrame(data={"model":list(),
                                                "accuracy":list(),
                                                "media precision":list(),
                                                "media recall":list(),
                                                "media f1-score":list(),
                                                "media ponderada precision":list(),
                                                "media ponderada recall":list(),
                                                "media ponderada f1-score":list(),
                                                "tempo fit":list(),
                                                "tempo predict":list()})

print(testX.shape)
print(testY.shape) 
print(type(dataframe_metricas_modelos))
dataframe_metricas_modelos

(5436, 512)
(5436,)
<class 'pandas.core.frame.DataFrame'>


,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict


# Avaliação dos Modelos

[classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report)


In [5]:
def avalia_lista_modelos(lista_modelos,lista_tempos,testX, testY):
    zip_modelo_tempo = list(zip(lista_modelos,lista_tempos))
    tabela_metricas_modelos = {"model":list(),
                               "accuracy":list(),
                               "media precision":list(),
                               "media recall":list(),
                               "media f1-score":list(),
                               "media ponderada precision":list(),
                               "media ponderada recall":list(),
                               "media ponderada f1-score":list(),
                               "tempo fit":list(),
                               "tempo predict":list()}
#     print(f"Lista de modelos = {list(lista_modelos)}")
    contador_para_nome_do_modelo = 1
    for model, tempo in zip_modelo_tempo:
#         print(f"Iniciando modelo {str(type(model))}")
        try:
            nome_modelo = f"{str(type(model)).split('.')[3][:-2]} {contador_para_nome_do_modelo}"
        except:
            nome_modelo = f"{str(type(model)).split('.')[2][:-2]} {contador_para_nome_do_modelo}"
        contador_para_nome_do_modelo+=1
        try:
            #Se tiver dado algum problema na hora de treinar_modelos, então tempo vai ser zero
            if tempo == 0:
                raise Exception("Propagando erro no treinamento!!")
            teste, tempo_predict = Avaliando_modelo(model, nome_modelo, None, testX, testY)
            tabela_metricas_modelos["model"].append(nome_modelo[:-2])
            tabela_metricas_modelos["accuracy"].append(teste["accuracy"])
            tabela_metricas_modelos["media precision"].append(teste["macro avg"]["precision"])
            tabela_metricas_modelos["media recall"].append(teste["macro avg"]["recall"])
            tabela_metricas_modelos["media f1-score"].append(teste["macro avg"]["f1-score"])
            tabela_metricas_modelos["media ponderada precision"].append(teste["weighted avg"]["precision"])
            tabela_metricas_modelos["media ponderada recall"].append(teste["weighted avg"]["recall"])
            tabela_metricas_modelos["media ponderada f1-score"].append(teste["weighted avg"]["f1-score"])
            tabela_metricas_modelos["tempo fit"].append(tempo)
            tabela_metricas_modelos["tempo predict"].append(tempo_predict)
            print(f"accuracy = {teste['accuracy']} e predict demorou {round(tempo_predict,2)} segundos \n")
            
        except:
            tabela_metricas_modelos["model"].append(nome_modelo)
            tabela_metricas_modelos["accuracy"].append(np.nan)
            tabela_metricas_modelos["media precision"].append(np.nan)
            tabela_metricas_modelos["media recall"].append(np.nan)
            tabela_metricas_modelos["media f1-score"].append(np.nan)
            tabela_metricas_modelos["media ponderada precision"].append(np.nan)
            tabela_metricas_modelos["media ponderada recall"].append(np.nan)
            tabela_metricas_modelos["media ponderada f1-score"].append(np.nan)
            tabela_metricas_modelos["tempo fit"].append(np.nan)
            tabela_metricas_modelos["tempo predict"].append(np.nan)
            
#         print(f"Finalizando modelo {str(type(model))}")
#     print(f"qtd de modelos = {len(list(tabela_metricas_modelos['model']))}\n{tabela_metricas_modelos}")
    dataframe_metricas = pd.DataFrame(data=tabela_metricas_modelos, index = [ind+1 for ind in range(len(list(tabela_metricas_modelos['model'])))])
    return dataframe_metricas

In [6]:
lb = label_transformer.classes_ # ["without_mask","mask_weared_incorrect","with_mask"]
def Avaliando_modelo(model, nome_modelo, NWHead = None,x_test=testX, y_test=testY):
    print(f"Avaliando modelo {nome_modelo} ....")
    tempo_predict =None
    try:
        inicio = time.time()
        predIdxs = model.predict(x_test, batch_size=32)
        fim = time.time()
        tempo_predict = fim-inicio
    except:
        inicio = time.time()
        predIdxs = model.predict(x_test)
        fim = time.time()
        tempo_predict = fim-inicio
    # for each image in the testing set we need to find the index of the
    # label with corresponding largest predicted probability
#     print(f"predIdxs = {type(predIdxs)} {predIdxs.dtype} shape = {predIdxs.shape}")
#     print(f"y_test = {type(y_test)} {y_test.dtype} shape = {y_test.shape}")
    #predIdxs = np.argmax(predIdxs, axis=1)
    #y_test = np.argmax(y_test,axis=1)
    
#     try:
#         print(f"parametros = {model.get_params()}")
#     except:
#         pass
    # show a nicely formatted classification report
    classification_report_val = classification_report(y_test,
                                                      predIdxs,
                                                      target_names=lb,
                                                      zero_division=0,
                                                      output_dict=True)
#     print(classification_report_val)

    # serialize the model to disk
#     print("[INFO] saving mask detector model...")

    if NWHead:
        # plot the training loss and accuracy
        N = EPOCHS
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(np.arange(0, N), NWHead.history["loss"], label="train_loss")
        plt.plot(np.arange(0, N), NWHead.history["val_loss"], label="val_loss")
        plt.plot(np.arange(0, N), NWHead.history["accuracy"], label="train_acc")
        plt.plot(np.arange(0, N), NWHead.history["val_accuracy"], label="val_acc")
        plt.title("Training Loss and Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend(loc="lower left")
        plt.show()
    return classification_report_val, tempo_predict

"""
UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 
                        Use `zero_division` parameter to control this behavior.
Este aviso acima acontece porque o modelo esperava prever 3 classes e acabou prevendo somente 2 classes. 
Na tabela vai aparecer a metrica 0, porem este alerta é anunciado. Para resolver isso é só colocar o parametro 
zero_division=0 dentro do método classification_report.

"""

'\nUndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. \n                        Use `zero_division` parameter to control this behavior.\nEste aviso acima acontece porque o modelo esperava prever 3 classes e acabou prevendo somente 2 classes. \nNa tabela vai aparecer a metrica 0, porem este alerta é anunciado. Para resolver isso é só colocar o parametro \nzero_division=0 dentro do método classification_report.\n\n'

# Modelos

In [7]:
# def treinar_modelos(lista_modelos,lista_tempos):
def treinar_modelos(lista_modelos):
    lista_tempos = list(range(len(lista_modelos)))
    for i in range(len(lista_modelos)):
        try:
            nome_modelo = f"{str(type(lista_modelos[i])).split('.')[3][:-2]} {i+1}"
        except:
            nome_modelo = f"{str(type(lista_modelos[i])).split('.')[2][:-2]} {i+1}"
        
        print(f"treinando {nome_modelo} . . . ")
        inicio = time.time()
        try:
            aux = lista_modelos[i]
            lista_modelos[i] = lista_modelos[i].fit(trainX, trainY)
            fim = time.time()
            lista_tempos[i] = fim-inicio
            if lista_tempos[i] <60:
                print(f"demorou {round(lista_tempos[i],2)} segundos \n")#\ninicio = {inicio}\nfim = {fim}\n")
            else:
                print(f"demorou {round(lista_tempos[i]/60,2)} minutos \n")#\ninicio = {inicio}\nfim = {fim}\n")
                
        except:
            print(f"Falha ao treinar {nome_modelo} . . . ")
            lista_tempos[i] = 0
        
            
    print("done!! :)")
    return lista_modelos,lista_tempos

## Support Vector Machines

In [8]:
from sklearn import svm

#### [Documentação LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)

#### [Documentação SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [9]:
# models_svm = (#svm.SVC(kernel='linear',cache_size=10000,decision_function_shape='ovo',max_iter=10000),
#               svm.LinearSVC(max_iter=10000, verbose=1),
# #               svm.SVC(kernel='rbf', gamma=0.7,cache_size=10000,decision_function_shape='ovo',max_iter=10000),
#               svm.SVC(kernel='poly', degree=3, gamma='auto',cache_size=10000,decision_function_shape='ovo',max_iter=1000, verbose=True),
# #               svm.SVC(C=1000, kernel='linear',cache_size=10000,decision_function_shape='ovo',max_iter=10000),
#               svm.LinearSVC(C=1000, max_iter=10000, verbose=1),
# #               svm.SVC(C=1000, kernel='rbf', gamma=0.7,cache_size=10000,decision_function_shape='ovo',max_iter=1000),
#               svm.SVC(C=1000, kernel='poly', degree=3, gamma='auto',cache_size=10000,decision_function_shape='ovo',max_iter=1000, verbose=True)
#              )
# #C=1 não rodou, decision_function_shape 'ovr' tbm não, cache_size 200 tbm não
# svm1 = svm.SVC(C=1, kernel='linear',cache_size=10000,decision_function_shape='ovo',max_iter=1000), 
# # testar com max_iter para não entrar em loop

# #LinearSVC == ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
# #outros não rodaram, ficaram rodando infinito

svm1 = svm.LinearSVC(max_iter=4000)
svm2 = svm.SVC(kernel='poly', 
               degree=3, 
               gamma='auto',
               cache_size=10000,
               decision_function_shape='ovo',
               max_iter=4000)
svm3 = svm.LinearSVC(C=1000, 
                     max_iter=4000)
svm4 = svm.SVC(C=1000, 
               kernel='poly', 
               degree=3, 
               gamma='auto',
               cache_size=10000,
               decision_function_shape='ovo',
               max_iter=4000)

svm5 = svm.SVC(C=1000, 
               cache_size=10000, 
               decision_function_shape='ovo', 
               kernel='poly',
               max_iter=1000, 
               random_state=42, 
               tol=0.0001)

In [10]:
models_svm_fit = [svm1,
                  svm2,
                  svm3,
                  svm4,
                  svm5]

models_svm_fit, models_svm_fit_time = treinar_modelos(models_svm_fit)

treinando LinearSVC 1 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


demorou 1.51 minutos 

treinando SVC 2 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\svm\_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


demorou 1.84 minutos 

treinando LinearSVC 3 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


demorou 1.52 minutos 

treinando SVC 4 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\svm\_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=4000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


demorou 1.84 minutos 

treinando SVC 5 . . . 
demorou 39.85 segundos 

done!! :)


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\svm\_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [11]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos, 
                                        avalia_lista_modelos(models_svm_fit,
                                                             models_svm_fit_time,
                                                             testX, 
                                                             testY)], 
                                        axis=0)
# dataframe_metricas_modelos.dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar)} segundos")

Avaliando modelo LinearSVC 1 ....
accuracy = 0.7910228108903605 e predict demorou 0.03 segundos 

Avaliando modelo SVC 2 ....
accuracy = 0.8281824871228844 e predict demorou 12.59 segundos 

Avaliando modelo LinearSVC 3 ....
accuracy = 0.7141280353200883 e predict demorou 0.02 segundos 

Avaliando modelo SVC 4 ....
accuracy = 0.8281824871228844 e predict demorou 12.66 segundos 

Avaliando modelo SVC 5 ....
accuracy = 0.7356512141280354 e predict demorou 9.64 segundos 

tempo para rodar toda a planilha é aproximadamente 442.0 segundos


## Stochastic Gradient Descent

In [12]:
from sklearn.linear_model import SGDClassifier

#### [Documentação SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)

In [13]:
sgd_clf1 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="hinge", penalty = "l2")
sgd_clf2 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="log", penalty = "l2")
sgd_clf3 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="modified_huber", penalty = "l2")
sgd_clf4 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="squared_hinge", penalty = "l2")
sgd_clf5 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="perceptron", penalty = "l2")

sgd_clf6 = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, loss="hinge", penalty = "l1")
sgd_clf7 = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, loss="log", penalty = "l1")
sgd_clf8 = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, loss="modified_huber", penalty = "l1")
sgd_clf9 = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, loss="squared_hinge", penalty = "l1")
sgd_clf10 = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, loss="perceptron", penalty = "l1")

sgd_clf11 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="hinge", penalty = "elasticnet")
sgd_clf12 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="log", penalty = "elasticnet")
sgd_clf13 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="modified_huber", penalty = "elasticnet")
sgd_clf14 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="squared_hinge", penalty = "elasticnet")
sgd_clf15 = SGDClassifier(random_state=42, max_iter=10000, tol=1e-3, loss="perceptron", penalty = "elasticnet")

sgd_clf16 = SGDClassifier(alpha=0.31622776601683794, max_iter=100, random_state=42,tol=0.0001, loss="hinge", penalty = "l2")
sgd_clf17 = SGDClassifier(alpha=0.0005298316906283707, max_iter=100, random_state=42,tol=0.0001, loss="squared_hinge", penalty = "elasticnet")

In [14]:
models_SGD_fit = [sgd_clf1
                  ,sgd_clf2
                  ,sgd_clf3
                  ,sgd_clf4
                  ,sgd_clf5
                  ,sgd_clf6
                  ,sgd_clf7
                  ,sgd_clf8
                  ,sgd_clf9
                  ,sgd_clf10
                  ,sgd_clf11
                  ,sgd_clf12
                  ,sgd_clf13
                  ,sgd_clf14
                  ,sgd_clf15
                  ,sgd_clf16
                  ,sgd_clf17]

models_SGD_fit, models_SGD_fit_time = treinar_modelos(models_SGD_fit)

treinando SGDClassifier 1 . . . 
demorou 8.1 segundos 

treinando SGDClassifier 2 . . . 
demorou 7.16 segundos 

treinando SGDClassifier 3 . . . 
demorou 6.06 segundos 

treinando SGDClassifier 4 . . . 
demorou 6.01 segundos 

treinando SGDClassifier 5 . . . 
demorou 6.69 segundos 

treinando SGDClassifier 6 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


demorou 59.31 segundos 

treinando SGDClassifier 7 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


demorou 58.24 segundos 

treinando SGDClassifier 8 . . . 


...

C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


demorou 59.23 segundos 

treinando SGDClassifier 11 . . . 
demorou 21.5 segundos 

treinando SGDClassifier 12 . . . 
demorou 17.03 segundos 

treinando SGDClassifier 13 . . . 
demorou 22.51 segundos 

treinando SGDClassifier 14 . . . 
demorou 18.49 segundos 

treinando SGDClassifier 15 . . . 
demorou 20.43 segundos 

treinando SGDClassifier 16 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


demorou 4.03 segundos 

treinando SGDClassifier 17 . . . 
demorou 10.93 segundos 

done!! :)


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [15]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos, 
                                        avalia_lista_modelos(models_SGD_fit, 
                                                             models_SGD_fit_time,
                                                             testX, 
                                                             testY)], 
                                       axis=0)
# dataframe_metricas_modelos.tail(len(models_SGD_fit)).dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos.tail(len(models_SGD_fit))["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar)} segundos")


Avaliando modelo SGDClassifier 1 ....
accuracy = 0.8211920529801324 e predict demorou 0.01 segundos 

Avaliando modelo SGDClassifier 2 ....
accuracy = 0.7763061074319353 e predict demorou 0.02 segundos 

Avaliando modelo SGDClassifier 3 ....
accuracy = 0.7454010301692421 e predict demorou 0.01 segundos 

Avaliando modelo SGDClassifier 4 ....
accuracy = 0.7844002943340692 e predict demorou 0.01 segundos 

Avaliando modelo SGDClassifier 5 ....
accuracy = 0.7888153053715967 e predict demorou 0.01 segundos 

Avaliando modelo SGDClassifier 6 ....
accuracy = 0.8083149374540103 e predict demorou 0.01 segundos 

Avaliando modelo SGDClassifier 7 ....
accuracy = 0.8040838852097131 e predict demorou 0.01 segundos 

Avaliando modelo SGDClassifier 8 ....
accuracy = 0.8112582781456954 e predict demorou 0.0 segundos 

Avaliando modelo SGDClassifier 9 ....
accuracy = 0.8145695364238411 e predict demorou 0.02 segundos 

Avaliando modelo SGDClassifier 10 ....
accuracy = 0.7798013245033113 e predict demo

## Decision Trees

In [16]:
from sklearn.tree import DecisionTreeClassifier

#### [Documentação](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

In [17]:
decision_tree1 = DecisionTreeClassifier(random_state=42, max_depth=2500, criterion = "gini", max_features = None)
decision_tree2 = DecisionTreeClassifier(random_state=42, max_depth=2500, criterion = "gini", max_features = "sqrt")
decision_tree3 = DecisionTreeClassifier(random_state=42, max_depth=2500, criterion = "gini", max_features = "log2")

decision_tree4 = DecisionTreeClassifier(random_state=42, max_depth=2500, criterion = "entropy", max_features = None)
decision_tree5 = DecisionTreeClassifier(random_state=42, max_depth=2500, criterion = "entropy", max_features = "sqrt")
decision_tree6 = DecisionTreeClassifier(random_state=42, max_depth=2500, criterion = "entropy", max_features = "log2")

In [18]:
models_decision_tree_fit = [decision_tree1,
                            decision_tree2,
                            decision_tree3,
                            decision_tree4,
                            decision_tree5,
                            decision_tree6]

models_decision_tree_fit, models_decision_tree_fit_time = treinar_modelos(models_decision_tree_fit)

treinando DecisionTreeClassifier 1 . . . 
demorou 10.83 segundos 

treinando DecisionTreeClassifier 2 . . . 
demorou 0.43 segundos 

treinando DecisionTreeClassifier 3 . . . 
demorou 0.17 segundos 

treinando DecisionTreeClassifier 4 . . . 
demorou 10.55 segundos 

treinando DecisionTreeClassifier 5 . . . 
demorou 0.54 segundos 

treinando DecisionTreeClassifier 6 . . . 
demorou 0.26 segundos 

done!! :)


In [19]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos, 
                                        avalia_lista_modelos(models_decision_tree_fit, 
                                                             models_decision_tree_fit_time,
                                                             testX, 
                                                             testY)], 
                                       axis=0)
# dataframe_metricas_modelos.tail(len(models_decision_tree_fit)).dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos.tail(len(models_decision_tree_fit))["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar)} segundos")

Avaliando modelo DecisionTreeClassifier 1 ....
accuracy = 0.7218543046357616 e predict demorou 0.02 segundos 

Avaliando modelo DecisionTreeClassifier 2 ....
accuracy = 0.6872700515084621 e predict demorou 0.02 segundos 

Avaliando modelo DecisionTreeClassifier 3 ....
accuracy = 0.6574687270051508 e predict demorou 0.0 segundos 

Avaliando modelo DecisionTreeClassifier 4 ....
accuracy = 0.7196467991169978 e predict demorou 0.01 segundos 

Avaliando modelo DecisionTreeClassifier 5 ....
accuracy = 0.6903973509933775 e predict demorou 0.02 segundos 

Avaliando modelo DecisionTreeClassifier 6 ....
accuracy = 0.6624356144223694 e predict demorou 0.02 segundos 

tempo para rodar toda a planilha é aproximadamente 23.0 segundos


## Naive Bayes

In [20]:
from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB

#### [Documentação GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)

#### [Documentação ComplementNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html#sklearn.naive_bayes.ComplementNB)

#### [Documentação MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)

In [21]:
models_NB1 = GaussianNB()
models_NB2 = MultinomialNB(fit_prior = True)
models_NB3 = ComplementNB(fit_prior = True)
models_NB4 = MultinomialNB(fit_prior = False)
models_NB5 = ComplementNB(fit_prior = False)



In [22]:
models_NB_fit = [models_NB1,
                 models_NB2,
                 models_NB3,
                 models_NB4,
                 models_NB5]


models_NB_fit, models_NB_fit_time = treinar_modelos(models_NB_fit)

treinando GaussianNB 1 . . . 
demorou 0.21 segundos 

treinando MultinomialNB 2 . . . 
demorou 0.01 segundos 

treinando ComplementNB 3 . . . 
demorou 0.03 segundos 

treinando MultinomialNB 4 . . . 
demorou 0.04 segundos 

treinando ComplementNB 5 . . . 
demorou 0.03 segundos 

done!! :)


In [23]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos,
                                        avalia_lista_modelos(models_NB_fit, 
                                                             models_NB_fit_time,
                                                             testX, 
                                                             testY)], 
                                       axis=0)
# dataframe_metricas_modelos.tail(len(models_NB_fit)).dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos.tail(len(models_NB_fit))["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar)} segundos")

Avaliando modelo GaussianNB 1 ....
accuracy = 0.6547093451066961 e predict demorou 0.15 segundos 

Avaliando modelo MultinomialNB 2 ....
accuracy = 0.655813097866078 e predict demorou 0.0 segundos 

Avaliando modelo ComplementNB 3 ....
accuracy = 0.6714495952906548 e predict demorou 0.01 segundos 

Avaliando modelo MultinomialNB 4 ....
accuracy = 0.655813097866078 e predict demorou 0.01 segundos 

Avaliando modelo ComplementNB 5 ....
accuracy = 0.6714495952906548 e predict demorou 0.01 segundos 

tempo para rodar toda a planilha é aproximadamente 0.0 segundos


## k-nearest neighbors algorithm

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

#### [Documentação RadiusNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsClassifier.html#sklearn.neighbors.RadiusNeighborsClassifier)

#### [Documentação KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)

In [66]:
KNN1 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1)
KNN2 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1)

KNN3 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 1)
KNN4 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 1)

KNN5 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1)
KNN6 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1)

KNN7 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2)
KNN8 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2)

KNN9 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 2)
KNN10 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 2)

KNN11 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2)
KNN12 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2)


KNN13 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "euclidean")
KNN14 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "euclidean")

KNN15 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 1, metric = "euclidean")
KNN16 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 1, metric = "euclidean")

KNN17 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "euclidean")
KNN18 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "euclidean")

KNN19 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "euclidean")
KNN20 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "euclidean")

KNN21 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 2, metric = "euclidean")
KNN22 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 2, metric = "euclidean")

KNN23 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "euclidean")
KNN24 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "euclidean")


KNN25 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "manhattan")
KNN26 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "manhattan")

KNN27 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 1, metric = "manhattan")
KNN28 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 1, metric = "manhattan")

KNN29 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "manhattan")
KNN30 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "manhattan")

KNN31 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "manhattan")
KNN32 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "manhattan")

KNN33 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 2, metric = "manhattan")
KNN34 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 2, metric = "manhattan")

KNN35 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "manhattan")
KNN36 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "manhattan")


KNN37 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "chebyshev")
KNN38 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "chebyshev")

KNN39 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 1, metric = "chebyshev")
KNN40 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 1, metric = "chebyshev")

KNN41 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "chebyshev")
KNN42 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "chebyshev")

KNN43 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "chebyshev")
KNN44 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "chebyshev")

KNN45 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 2, metric = "chebyshev")
KNN46 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 2, metric = "chebyshev")

KNN47 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "chebyshev")
KNN48 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "chebyshev")


KNN49 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})
KNN50 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})

KNN51 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})
KNN52 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})

KNN53 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})
KNN54 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})

KNN55 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})
KNN56 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "mahalanobis", metric_params={'V': np.cov(trainX)})


dist = lambda x,y: DistanceMetric.get_metric('Wminkowski').pairwise(x,y,2)

KNN57 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = dist)
KNN58 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = dist)

KNN59 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = dist)
KNN60 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = dist)

KNN61 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = dist)
KNN62 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = dist)

KNN63 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = dist)
KNN64 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = dist)


KNN65 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "cityblock")
KNN66 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "cityblock")

KNN67 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 1, metric = "cityblock")
KNN68 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 1, metric = "cityblock")

KNN69 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "cityblock")
KNN70 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "cityblock")

KNN71 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "cityblock")
KNN72 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "cityblock")

KNN73 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "kd_tree", p= 2, metric = "cityblock")
KNN74 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "kd_tree", p= 2, metric = "cityblock")

KNN75 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "cityblock")
KNN76 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "cityblock")



KNN77 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "canberra")
KNN78 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "canberra")

KNN79 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "canberra")
KNN80 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "canberra")

KNN81 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "canberra")
KNN82 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "canberra")

KNN83 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "canberra")
KNN84 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "canberra")



KNN85 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "braycurtis")
KNN86 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "braycurtis")

KNN87 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "braycurtis")
KNN88 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "braycurtis")

KNN89 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "braycurtis")
KNN90 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "braycurtis")

KNN91 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "braycurtis")
KNN92 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "braycurtis")

KNN93 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 1, metric = "hamming")
KNN94 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 1, metric = "hamming")

KNN95 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 1, metric = "hamming")
KNN96 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 1, metric = "hamming")

KNN97 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "ball_tree", p= 2, metric = "hamming")
KNN98 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "ball_tree", p= 2, metric = "hamming")

KNN99 = KNeighborsClassifier(n_neighbors=3, weights="uniform", algorithm= "brute", p= 2, metric = "hamming")
KNN100 = KNeighborsClassifier(n_neighbors=3, weights="distance", algorithm= "brute", p= 2, metric = "hamming")


In [67]:
models_KNN_fit =[KNN1,
                 KNN2,
                 KNN3,
                 KNN4,
                 KNN5,
                 KNN6,
                 KNN7,
                 KNN8,
                 KNN9,
                 KNN10,
                 KNN11,
                 KNN12,
                 KNN13,
                 KNN14,
                 KNN15,
                 KNN16,
                 KNN17,
                 KNN18,
                 KNN19,
                 KNN20,
                 KNN21,
                 KNN22,
                 KNN23,
                 KNN24,
                 KNN25,
                 KNN26,
                 KNN27,
                 KNN28,
                 KNN29,
                 KNN30,
                 KNN31,
                 KNN32,
                 KNN33,
                 KNN34,
                 KNN35,
                 KNN36,
                 KNN37,
                 KNN38,
                 KNN39,
                 KNN40,
                 KNN41,
                 KNN42,
                 KNN43,
                 KNN44,
                 KNN45,
                 KNN46,
                 KNN47,
                 KNN48,
                 KNN49,
                 KNN50,
                 KNN51,
                 KNN52,
                 KNN53,
                 KNN54,
                 KNN55,
                 KNN56,
                 KNN57,
                 KNN58,
                 KNN59,
                 KNN60,
                 KNN61,
                 KNN62,
                 KNN63,
                 KNN64,
                 KNN65,
                 KNN66,
                 KNN67,
                 KNN68,
                 KNN69,
                 KNN70,
                 KNN71,
                 KNN72,
                 KNN73,
                 KNN74,
                 KNN75,
                 KNN76,
                 KNN77,
                 KNN78,
                 KNN79,
                 KNN80,
                 KNN81,
                 KNN82,
                 KNN83,
                 KNN84,
                 KNN85,
                 KNN86,
                 KNN87,
                 KNN88,
                 KNN89,
                 KNN90,
                 KNN91,
                 KNN92,
                 KNN93,
                 KNN94,
                 KNN95,
                 KNN96,
                 KNN97,
                 KNN98,
                 KNN99,
                 KNN100]#,
#                  KNN101,
#                  KNN102,
#                  KNN103,
#                  KNN104,
#                  KNN105,
#                  KNN106,
#                  KNN107,
#                  KNN108,
#                  KNN109,
#                  KNN110,
#                  KNN111,
#                  KNN112,
#                  KNN113,
#                  KNN114,
#                  KNN115,
#                  KNN116,
#                  KNN117,
#                  KNN118,
#                  KNN119,
#                  KNN120,
#                  KNN121,
#                  KNN122,
#                  KNN123,
#                  KNN124,
#                  KNN125,
#                  KNN126,
#                  KNN127,
#                  KNN128,
#                  KNN129,
#                  KNN130,
#                  KNN131,
#                  KNN132]


models_KNN_fit, models_KNN_fit_time = treinar_modelos(models_KNN_fit)

treinando KNeighborsClassifier 1 . . . 
demorou 2.59 segundos 

treinando KNeighborsClassifier 2 . . . 
demorou 2.65 segundos 

treinando KNeighborsClassifier 3 . . . 
demorou 3.54 segundos 

treinando KNeighborsClassifier 4 . . . 
demorou 3.6 segundos 

treinando KNeighborsClassifier 5 . . . 
demorou 0.03 segundos 

treinando KNeighborsClassifier 6 . . . 
demorou 0.03 segundos 

treinando KNeighborsClassifier 7 . . . 
demorou 2.62 segundos 

treinando KNeighborsClassifier 8 . . . 
demorou 2.56 segundos 

treinando KNeighborsClassifier 9 . . . 
demorou 3.6 segundos 

treinando KNeighborsClassifier 10 . . . 
demorou 3.63 segundos 

treinando KNeighborsClassifier 11 . . . 
demorou 0.03 segundos 

treinando KNeighborsClassifier 12 . . . 
demorou 0.03 segundos 

treinando KNeighborsClassifier 13 . . . 
demorou 2.55 segundos 

treinando KNeighborsClassifier 14 . . . 
demorou 2.65 segundos 

treinando KNeighborsClassifier 15 . . . 
demorou 3.54 segundos 

treinando KNeighborsClassifier 16 . 

In [68]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos,
                                        avalia_lista_modelos(models_KNN_fit, 
                                                             models_KNN_fit_time,
                                                             testX, 
                                                             testY)], 
                                       axis=0)
# dataframe_metricas_modelos.tail(len(models_KNN_fit)).dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos.tail(len(models_KNN_fit))["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar)} segundos")

Avaliando modelo KNeighborsClassifier 1 ....
accuracy = 0.858167770419426 e predict demorou 58.04 segundos 

Avaliando modelo KNeighborsClassifier 2 ....
accuracy = 0.8655261221486387 e predict demorou 57.52 segundos 

Avaliando modelo KNeighborsClassifier 3 ....
accuracy = 0.858167770419426 e predict demorou 68.76 segundos 

Avaliando modelo KNeighborsClassifier 4 ....
accuracy = 0.8655261221486387 e predict demorou 68.46 segundos 

Avaliando modelo KNeighborsClassifier 5 ....
accuracy = 0.858167770419426 e predict demorou 51.07 segundos 

Avaliando modelo KNeighborsClassifier 6 ....
accuracy = 0.8655261221486387 e predict demorou 50.99 segundos 

Avaliando modelo KNeighborsClassifier 7 ....
accuracy = 0.8568800588668138 e predict demorou 59.49 segundos 

Avaliando modelo KNeighborsClassifier 8 ....
accuracy = 0.8657100809418691 e predict demorou 59.15 segundos 

Avaliando modelo KNeighborsClassifier 9 ....
accuracy = 0.8568800588668138 e predict demorou 81.48 segundos 

Avaliando mod

C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\metrics\pairwise.py:1462: FutureWarning: from version 0.25, pairwise_distances for metric='mahalanobis' will require VI to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


accuracy = 0.4300956585724798 e predict demorou 24618.47 segundos 

Avaliando modelo KNeighborsClassifier 52 ....


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\metrics\pairwise.py:1462: FutureWarning: from version 0.25, pairwise_distances for metric='mahalanobis' will require VI to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


accuracy = 0.43414275202354674 e predict demorou 24385.11 segundos 

Avaliando modelo KNeighborsClassifier 55 ....


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\metrics\pairwise.py:1462: FutureWarning: from version 0.25, pairwise_distances for metric='mahalanobis' will require VI to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


accuracy = 0.4300956585724798 e predict demorou 24424.86 segundos 

Avaliando modelo KNeighborsClassifier 56 ....


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\metrics\pairwise.py:1462: FutureWarning: from version 0.25, pairwise_distances for metric='mahalanobis' will require VI to be specified if Y is passed.
  "specified if Y is passed.", FutureWarning)


accuracy = 0.43414275202354674 e predict demorou 24443.65 segundos 

Avaliando modelo KNeighborsClassifier 59 ....
Avaliando modelo KNeighborsClassifier 60 ....
Avaliando modelo KNeighborsClassifier 63 ....
Avaliando modelo KNeighborsClassifier 64 ....
Avaliando modelo KNeighborsClassifier 65 ....
accuracy = 0.858167770419426 e predict demorou 57.76 segundos 

Avaliando modelo KNeighborsClassifier 66 ....
accuracy = 0.8655261221486387 e predict demorou 57.81 segundos 

Avaliando modelo KNeighborsClassifier 67 ....
accuracy = 0.858167770419426 e predict demorou 68.54 segundos 

Avaliando modelo KNeighborsClassifier 68 ....
accuracy = 0.8655261221486387 e predict demorou 68.35 segundos 

Avaliando modelo KNeighborsClassifier 69 ....
accuracy = 0.858167770419426 e predict demorou 51.16 segundos 

Avaliando modelo KNeighborsClassifier 70 ....
accuracy = 0.8655261221486387 e predict demorou 51.08 segundos 

Avaliando modelo KNeighborsClassifier 71 ....
accuracy = 0.858167770419426 e predict

## LogisticRegression

In [69]:
from sklearn.linear_model import LogisticRegression

#### [Documentação](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)

In [70]:
RegLog1 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "l2", C= 1)
RegLog2 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "l2", C= 1)
RegLog3 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l2", C= 1)
RegLog4 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "l2", C= 1)
RegLog5 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "l2", C= 1)

RegLog6 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "l1", C= 1)
RegLog7 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "l1", C= 1)
RegLog8 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l1", C= 1)
RegLog9 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "l1", C= 1)
RegLog10 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "l1", C= 1)

RegLog11 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "elasticnet", C= 1)
RegLog12 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "elasticnet", C= 1)
RegLog13 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "elasticnet", C= 1)
RegLog14 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "elasticnet", C= 1)
RegLog15 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "elasticnet", C= 1)

RegLog16 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "none", C= 1)
RegLog17 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "none", C= 1)
RegLog18 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "none", C= 1)
RegLog19 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "none", C= 1)
RegLog20 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "none", C= 1)



RegLog21 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "l2", C= 100)
RegLog22 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "l2", C= 100)
RegLog23 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l2", C= 100)
RegLog24 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "l2", C= 100)
RegLog25 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "l2", C= 100)

RegLog26 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "l1", C= 100)
RegLog27 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "l1", C= 100)
RegLog28 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l1", C= 100)
RegLog29 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "l1", C= 100)
RegLog30 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "l1", C= 100)

RegLog31 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "elasticnet", C= 100)
RegLog32 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "elasticnet", C= 100)
RegLog33 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "elasticnet", C= 100)
RegLog34 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "elasticnet", C= 100)
RegLog35 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "elasticnet", C= 100)

RegLog36 = LogisticRegression(random_state=0, max_iter = 1000, solver = "newton-cg", penalty= "none", C= 100)
RegLog37 = LogisticRegression(random_state=0, max_iter = 1000, solver = "lbfgs", penalty= "none", C= 100)
RegLog38 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "none", C= 100)
RegLog39 = LogisticRegression(random_state=0, max_iter = 1000, solver = "sag", penalty= "none", C= 100)
RegLog40 = LogisticRegression(random_state=0, max_iter = 1000, solver = "saga", penalty= "none", C= 100)

RegLog41 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l1", C= 1, class_weight = "balanced")
RegLog42 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l2", C= 1, class_weight = "balanced")

RegLog43 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l1", C= 200, class_weight = "balanced")
RegLog44 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l2", C= 200, class_weight = "balanced")

RegLog45 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l1", C= 200, class_weight = None)
RegLog46 = LogisticRegression(random_state=0, max_iter = 1000, solver = "liblinear", penalty= "l2", C= 200, class_weight = None)

In [71]:
RegLog_fit = [RegLog1, 
              RegLog2, 
              RegLog3, 
              RegLog4, 
              RegLog5, 
              RegLog6, 
              RegLog7, 
              RegLog8, 
              RegLog9, 
              RegLog10,
              RegLog11,
              RegLog12,
              RegLog13,
              RegLog14,
              RegLog15,
              RegLog16,
              RegLog17,
              RegLog18,
              RegLog19,
              RegLog20,
              RegLog21,
              RegLog22,
              RegLog23,
              RegLog24,
              RegLog25,
              RegLog26,
              RegLog27,
              RegLog28,
              RegLog29,
              RegLog30,
              RegLog31,
              RegLog32,
              RegLog33,
              RegLog34,
              RegLog35,
              RegLog36,
              RegLog37,
              RegLog38,
              RegLog39,
              RegLog40,
              RegLog41,
              RegLog42,
              RegLog43,
              RegLog44,
              RegLog45,
              RegLog46]

RegLog_fit, RegLog_fit_time = treinar_modelos(RegLog_fit)

treinando LogisticRegression 1 . . . 
demorou 15.51 segundos 

treinando LogisticRegression 2 . . . 
demorou 12.31 segundos 

treinando LogisticRegression 3 . . . 
demorou 21.77 segundos 

treinando LogisticRegression 4 . . . 
demorou 17.21 segundos 

treinando LogisticRegression 5 . . . 
demorou 38.6 segundos 

treinando LogisticRegression 6 . . . 
Falha ao treinar LogisticRegression 6 . . . 
treinando LogisticRegression 7 . . . 
Falha ao treinar LogisticRegression 7 . . . 
treinando LogisticRegression 8 . . . 
demorou 5.58 segundos 

treinando LogisticRegression 9 . . . 
Falha ao treinar LogisticRegression 9 . . . 
treinando LogisticRegression 10 . . . 
demorou 1.15 minutos 

treinando LogisticRegression 11 . . . 
Falha ao treinar LogisticRegression 11 . . . 
treinando LogisticRegression 12 . . . 
Falha ao treinar LogisticRegression 12 . . . 
treinando LogisticRegression 13 . . . 
Falha ao treinar LogisticRegression 13 . . . 
treinando LogisticRegression 14 . . . 
Falha ao treinar Lo

C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


demorou 12.72 segundos 

treinando LogisticRegression 18 . . . 
Falha ao treinar LogisticRegression 18 . . . 
treinando LogisticRegression 19 . . . 
demorou 17.52 segundos 

treinando LogisticRegression 20 . . . 
demorou 38.5 segundos 

treinando LogisticRegression 21 . . . 
demorou 14.31 segundos 

treinando LogisticRegression 22 . . . 
demorou 9.8 segundos 

treinando LogisticRegression 23 . . . 
demorou 23.22 segundos 

treinando LogisticRegression 24 . . . 
demorou 17.29 segundos 

treinando LogisticRegression 25 . . . 
demorou 38.56 segundos 

treinando LogisticRegression 26 . . . 
Falha ao treinar LogisticRegression 26 . . . 
treinando LogisticRegression 27 . . . 
Falha ao treinar LogisticRegression 27 . . . 
treinando LogisticRegression 28 . . . 
demorou 5.42 segundos 

treinando LogisticRegression 29 . . . 
Falha ao treinar LogisticRegression 29 . . . 
treinando LogisticRegression 30 . . . 
demorou 1.37 minutos 

treinando LogisticRegression 31 . . . 
Falha ao treinar LogisticR

C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


demorou 14.1 segundos 

treinando LogisticRegression 37 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting pe

demorou 12.6 segundos 

treinando LogisticRegression 38 . . . 
Falha ao treinar LogisticRegression 38 . . . 
treinando LogisticRegression 39 . . . 
demorou 17.27 segundos 

treinando LogisticRegression 40 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


demorou 38.39 segundos 

treinando LogisticRegression 41 . . . 
demorou 5.5 segundos 

treinando LogisticRegression 42 . . . 
demorou 18.35 segundos 

treinando LogisticRegression 43 . . . 
demorou 6.42 segundos 

treinando LogisticRegression 44 . . . 
demorou 19.01 segundos 

treinando LogisticRegression 45 . . . 
demorou 5.34 segundos 

treinando LogisticRegression 46 . . . 
demorou 18.46 segundos 

done!! :)


In [72]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos, 
                                        avalia_lista_modelos(RegLog_fit, 
                                                             RegLog_fit_time,
                                                             testX, 
                                                             testY)], axis=0)
# dataframe_metricas_modelos.tail(len(RegLog_fit)).dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos.tail(len(RegLog_fit))["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar/60,0)} minutos")

Avaliando modelo LogisticRegression 1 ....
accuracy = 0.8403237674760854 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegression 2 ....
accuracy = 0.8403237674760854 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegression 3 ....
accuracy = 0.8410596026490066 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegression 4 ....
accuracy = 0.8390360559234732 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegression 5 ....
accuracy = 0.8390360559234732 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegression 8 ....
accuracy = 0.8410596026490066 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegression 10 ....
accuracy = 0.8390360559234732 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegression 16 ....
accuracy = 0.8403237674760854 e predict demorou 0.02 segundos 

Avaliando modelo LogisticRegression 17 ....
accuracy = 0.8401398086828551 e predict demorou 0.01 segundos 

Avaliando modelo LogisticRegressio

## Neural network

In [73]:
from sklearn.neural_network import MLPClassifier

#### [Documentação](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)

In [74]:
clf_MLP1 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "relu")
clf_MLP2 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "relu")
clf_MLP3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "relu")

clf_MLP4 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.1, activation = "relu")
clf_MLP5 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.1, activation = "relu")
clf_MLP6 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.1, activation = "relu")

clf_MLP7 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "relu")
clf_MLP8 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "relu")
clf_MLP9 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "relu")



clf_MLP10 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.0001)
clf_MLP11 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.0001)
clf_MLP12 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.0001)

clf_MLP13 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.0001)
clf_MLP14 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.0001)
clf_MLP15 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.0001)

clf_MLP16 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.0001)
clf_MLP17 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.0001)
clf_MLP18 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.0001)




clf_MLP19 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "tanh")
clf_MLP20 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "tanh")
clf_MLP21 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "tanh")

clf_MLP22 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.1, activation = "tanh")
clf_MLP23 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.1, activation = "tanh")
clf_MLP24 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.1, activation = "tanh")

clf_MLP25 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "tanh")
clf_MLP26 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "tanh")
clf_MLP27 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "tanh")





clf_MLP28 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "logistic")
clf_MLP29 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "logistic")
clf_MLP30 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "logistic")

clf_MLP31 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.1, activation = "logistic")
clf_MLP32 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.1, activation = "logistic")
clf_MLP33 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.1, activation = "logistic")

clf_MLP34 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "logistic")
clf_MLP35 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "logistic")
clf_MLP36 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "logistic")



clf_MLP28 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "identity")
clf_MLP29 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "identity")
clf_MLP30 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "identity")

clf_MLP31 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.1, activation = "identity")
clf_MLP32 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.1, activation = "identity")
clf_MLP33 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 50, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.1, activation = "identity")

clf_MLP34 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="constant",learning_rate_init= 0.01, activation = "identity")
clf_MLP35 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="invscaling",learning_rate_init= 0.01, activation = "identity")
clf_MLP36 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(224, 100, 3), random_state=42, max_iter=200000,learning_rate="adaptive",learning_rate_init= 0.01, activation = "identity")


In [75]:
clf_MLP_fit =  [clf_MLP1,
                clf_MLP2,
                clf_MLP3,
                clf_MLP4,
                clf_MLP5,
                clf_MLP6,
                clf_MLP7,
                clf_MLP8,
                clf_MLP9,
                clf_MLP10,
                clf_MLP11,
                clf_MLP12,
                clf_MLP13,
                clf_MLP14,
                clf_MLP15,
                clf_MLP16,
                clf_MLP17,
                clf_MLP18,
                clf_MLP19,
                clf_MLP20,
                clf_MLP21,
                clf_MLP22,
                clf_MLP23,
                clf_MLP24,
                clf_MLP25,
                clf_MLP26,
                clf_MLP27,
                clf_MLP28,
                clf_MLP29,
                clf_MLP30,
                clf_MLP31,
                clf_MLP32,
                clf_MLP33,
                clf_MLP34,
                clf_MLP35,
                clf_MLP36]

clf_MLP_fit, clf_MLP_fit_time = treinar_modelos(clf_MLP_fit)

treinando MLPClassifier 1 . . . 
demorou 2.35 minutos 

treinando MLPClassifier 2 . . . 
demorou 2.36 minutos 

treinando MLPClassifier 3 . . . 
demorou 2.39 minutos 

treinando MLPClassifier 4 . . . 
demorou 4.06 minutos 

treinando MLPClassifier 5 . . . 
demorou 4.05 minutos 

treinando MLPClassifier 6 . . . 
demorou 4.05 minutos 

treinando MLPClassifier 7 . . . 
demorou 2.68 segundos 

treinando MLPClassifier 8 . . . 
demorou 2.57 segundos 

treinando MLPClassifier 9 . . . 
demorou 2.58 segundos 

treinando MLPClassifier 10 . . . 
demorou 2.36 minutos 

treinando MLPClassifier 11 . . . 
demorou 2.37 minutos 

treinando MLPClassifier 12 . . . 
demorou 2.39 minutos 

treinando MLPClassifier 13 . . . 
demorou 4.07 minutos 

treinando MLPClassifier 14 . . . 
demorou 4.07 minutos 

treinando MLPClassifier 15 . . . 
demorou 4.08 minutos 

treinando MLPClassifier 16 . . . 
demorou 2.48 segundos 

treinando MLPClassifier 17 . . . 
demorou 2.53 segundos 

treinando MLPClassifier 18 . . . 
d

C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


demorou 13.27 minutos 

treinando MLPClassifier 20 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


demorou 13.25 minutos 

treinando MLPClassifier 21 . . . 


C:\Users\wesle\anaconda3\envs\py3_6_tensorflow2_1\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


demorou 13.25 minutos 

treinando MLPClassifier 22 . . . 
demorou 12.98 minutos 

treinando MLPClassifier 23 . . . 
demorou 13.02 minutos 

treinando MLPClassifier 24 . . . 
demorou 13.05 minutos 

treinando MLPClassifier 25 . . . 
demorou 4.98 minutos 

treinando MLPClassifier 26 . . . 
demorou 4.95 minutos 

treinando MLPClassifier 27 . . . 
demorou 4.93 minutos 

treinando MLPClassifier 28 . . . 
demorou 1.47 minutos 

treinando MLPClassifier 29 . . . 
demorou 1.46 minutos 

treinando MLPClassifier 30 . . . 
demorou 1.48 minutos 

treinando MLPClassifier 31 . . . 
demorou 4.14 minutos 

treinando MLPClassifier 32 . . . 
demorou 4.12 minutos 

treinando MLPClassifier 33 . . . 
demorou 4.13 minutos 

treinando MLPClassifier 34 . . . 
demorou 3.33 minutos 

treinando MLPClassifier 35 . . . 
demorou 3.35 minutos 

treinando MLPClassifier 36 . . . 
demorou 3.33 minutos 

done!! :)


In [76]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos, 
                                        avalia_lista_modelos(clf_MLP_fit, 
                                                             clf_MLP_fit_time,
                                                             testX, 
                                                             testY)], 
                                       axis=0)
# dataframe_metricas_modelos.tail(len(clf_MLP_fit)).dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos.tail(len(clf_MLP_fit))["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar/60,0)} minutos")

Avaliando modelo MLPClassifier 1 ....
accuracy = 0.8631346578366446 e predict demorou 0.03 segundos 

Avaliando modelo MLPClassifier 2 ....
accuracy = 0.8631346578366446 e predict demorou 0.01 segundos 

Avaliando modelo MLPClassifier 3 ....
accuracy = 0.8631346578366446 e predict demorou 0.02 segundos 

Avaliando modelo MLPClassifier 4 ....
accuracy = 0.6455114054451803 e predict demorou 0.03 segundos 

Avaliando modelo MLPClassifier 5 ....
accuracy = 0.6455114054451803 e predict demorou 0.03 segundos 

Avaliando modelo MLPClassifier 6 ....
accuracy = 0.6455114054451803 e predict demorou 0.04 segundos 

Avaliando modelo MLPClassifier 7 ....
accuracy = 0.41096394407652687 e predict demorou 0.07 segundos 

Avaliando modelo MLPClassifier 8 ....
accuracy = 0.41096394407652687 e predict demorou 0.06 segundos 

Avaliando modelo MLPClassifier 9 ....
accuracy = 0.41096394407652687 e predict demorou 0.07 segundos 

Avaliando modelo MLPClassifier 10 ....
accuracy = 0.8631346578366446 e predict 

## Ensemble methods

In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [86]:
path = decision_tree1.cost_complexity_pruning_path(trainX, trainY)
ccp_alphas, impurities = path.ccp_alphas, path.impurities
len(ccp_alphas)

1150

#### [Documentação RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

#### [Documentação ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier)

#### [Documentação AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)

#### [Documentação GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)

In [87]:
# Ensemble_methods = (RandomForestClassifier(max_depth=100, random_state=42),
#                      ExtraTreesClassifier(n_estimators=100, random_state=42),
#                      AdaBoostClassifier(n_estimators=100, random_state=42),
#                      GradientBoostingClassifier(random_state=42))


Ensemble_methods1 = RandomForestClassifier(max_depth=1000, random_state=42, criterion= "gini", class_weight=None)
Ensemble_methods2 = RandomForestClassifier(max_depth=1000, random_state=42, criterion= "entropy", class_weight=None)

Ensemble_methods3 = RandomForestClassifier(max_depth=1000, random_state=42, criterion= "gini", class_weight = "balanced")
Ensemble_methods4 = RandomForestClassifier(max_depth=1000, random_state=42, criterion= "entropy", class_weight = "balanced")

Ensemble_methods5 = ExtraTreesClassifier(n_estimators=100, random_state=42, criterion= "gini", class_weight=None)
Ensemble_methods6 = ExtraTreesClassifier(n_estimators=100, random_state=42, criterion= "entropy", class_weight=None)

Ensemble_methods7 = ExtraTreesClassifier(n_estimators=100, random_state=42, criterion= "gini", class_weight = "balanced")
Ensemble_methods8 = ExtraTreesClassifier(n_estimators=100, random_state=42, criterion= "entropy", class_weight = "balanced")

Ensemble_methods9 = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm="SAMME", learning_rate= 1)
Ensemble_methods10 = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm="SAMME.R", learning_rate= 1)

Ensemble_methods11 = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm="SAMME", learning_rate= 2)
Ensemble_methods12 = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm="SAMME.R", learning_rate= 2)

Ensemble_methods13 = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm="SAMME", learning_rate= 0.5)
Ensemble_methods14 = AdaBoostClassifier(n_estimators=100, random_state=42, algorithm="SAMME.R", learning_rate= 0.5)

Ensemble_methods15 = GradientBoostingClassifier(random_state=42,n_iter_no_change=3, loss= "deviance", criterion= "friedman_mse", tol=1e-2)
Ensemble_methods16 = GradientBoostingClassifier(random_state=42,n_iter_no_change=3, loss= "deviance", criterion= "friedman_mse", tol=1e-4)
Ensemble_methods17 = GradientBoostingClassifier(random_state=42,n_iter_no_change=3, loss= "deviance", criterion= "friedman_mse", tol=1e-6)

Ensemble_methods18 = GradientBoostingClassifier(random_state=42,n_iter_no_change=3, loss= "deviance", criterion= "mse", tol=1e-2)
Ensemble_methods19 = GradientBoostingClassifier(random_state=42,n_iter_no_change=3, loss= "deviance", criterion= "mse", tol=1e-4)
Ensemble_methods20 = GradientBoostingClassifier(random_state=42,n_iter_no_change=3, loss= "deviance", criterion= "mse", tol=1e-6)

##Fica mmuito custoso utilizar a metrica MAE, demora muito e não é esperado que tenha um resultado melhor
# Ensemble_methods21 = GradientBoostingClassifier(random_state=42,n_iter_no_change=3, loss= "deviance", criterion= "mae")
# Ensemble_methods22 = GradientBoostingClassifier(random_state=42,n_iter_no_change=2, loss= "exponential", criterion= "mae")

In [88]:
models_Ensemble_methods_fit = [Ensemble_methods1,
                               Ensemble_methods2,
                               Ensemble_methods3,
                               Ensemble_methods4,
                               Ensemble_methods5,
                               Ensemble_methods6,
                               Ensemble_methods7,
                               Ensemble_methods8,
                               Ensemble_methods9,
                               Ensemble_methods10,
                               Ensemble_methods11,
                               Ensemble_methods12,
                               Ensemble_methods13,
                               Ensemble_methods14,
                               Ensemble_methods15,
                               Ensemble_methods16,
                               Ensemble_methods17,
                               Ensemble_methods18,
                               Ensemble_methods19,
                               Ensemble_methods20]

models_Ensemble_methods_fit, models_Ensemble_methods_fit_time = treinar_modelos(models_Ensemble_methods_fit)

treinando RandomForestClassifier 1 . . . 
demorou 20.48 segundos 

treinando RandomForestClassifier 2 . . . 
demorou 32.84 segundos 

treinando RandomForestClassifier 3 . . . 
demorou 22.35 segundos 

treinando RandomForestClassifier 4 . . . 
demorou 34.58 segundos 

treinando ExtraTreesClassifier 5 . . . 
demorou 8.06 segundos 

treinando ExtraTreesClassifier 6 . . . 
demorou 8.23 segundos 

treinando ExtraTreesClassifier 7 . . . 
demorou 8.44 segundos 

treinando ExtraTreesClassifier 8 . . . 
demorou 8.42 segundos 

treinando AdaBoostClassifier 9 . . . 
demorou 1.14 minutos 

treinando AdaBoostClassifier 10 . . . 
demorou 1.14 minutos 

treinando AdaBoostClassifier 11 . . . 
demorou 1.12 minutos 

treinando AdaBoostClassifier 12 . . . 
demorou 1.11 minutos 

treinando AdaBoostClassifier 13 . . . 
demorou 1.06 minutos 

treinando AdaBoostClassifier 14 . . . 
demorou 1.07 minutos 

treinando GradientBoostingClassifier 15 . . . 
demorou 2.76 minutos 

treinando GradientBoostingClassifie

In [89]:
dataframe_metricas_modelos = pd.concat([dataframe_metricas_modelos, 
                                        avalia_lista_modelos(models_Ensemble_methods_fit, 
                                                             models_Ensemble_methods_fit_time,
                                                             testX, 
                                                             testY)], 
                                       axis=0)
# dataframe_metricas_modelos.tail(len(models_Ensemble_methods_fit)).dropna().reset_index(drop=False)
tempo_para_rodar = dataframe_metricas_modelos.tail(len(models_Ensemble_methods_fit))["tempo fit"].sum()
print(f"tempo para rodar toda a planilha é aproximadamente {round(tempo_para_rodar/60,0)} minutos")

Avaliando modelo RandomForestClassifier 1 ....
accuracy = 0.8303899926416483 e predict demorou 0.21 segundos 

Avaliando modelo RandomForestClassifier 2 ....
accuracy = 0.8309418690213393 e predict demorou 0.18 segundos 

Avaliando modelo RandomForestClassifier 3 ....
accuracy = 0.8340691685062546 e predict demorou 0.19 segundos 

Avaliando modelo RandomForestClassifier 4 ....
accuracy = 0.8305739514348786 e predict demorou 0.16 segundos 

Avaliando modelo ExtraTreesClassifier 5 ....
accuracy = 0.8252391464311994 e predict demorou 0.2 segundos 

Avaliando modelo ExtraTreesClassifier 6 ....
accuracy = 0.8300220750551877 e predict demorou 0.2 segundos 

Avaliando modelo ExtraTreesClassifier 7 ....
accuracy = 0.8309418690213393 e predict demorou 0.2 segundos 

Avaliando modelo ExtraTreesClassifier 8 ....
accuracy = 0.8296541574687271 e predict demorou 0.21 segundos 

Avaliando modelo AdaBoostClassifier 9 ....
accuracy = 0.7593818984547461 e predict demorou 0.89 segundos 

Avaliando modelo

# Resultados

In [97]:
tempo_para_rodar_fit_total = dataframe_metricas_modelos["tempo fit"].sum()
tempo_para_rodar_predict_total = dataframe_metricas_modelos["tempo predict"].sum()
Tempo_treinamento_e_predict = (tempo_para_rodar_fit_total+tempo_para_rodar_predict_total)/60
print(f"tempo para treinar e predizer é aproximadamente {int(Tempo_treinamento_e_predict/60)} horas e {round(Tempo_treinamento_e_predict%60,1)} minutos")
fimGeral = time.time()

tempo para treinar e predizer é aproximadamente 32 horas e 30.2 minutos


In [98]:
tempo_Total = (fimGeral-inicioGeral)/60
print(tempo_Total)
print(f"tempo para rodar todo o Notebook é aproximadamente {int(tempo_Total/60)} horas e {round(tempo_Total%60,1)} minutos")

2713.0147833744686
tempo para rodar todo o Notebook é aproximadamente 45 horas e 13.0 minutos


In [99]:
#remove as linhas nulas e as que possuem acuracia abaixo de 60%
dataframe_metricas_modelos = dataframe_metricas_modelos.dropna()
dataframe_metricas_modelos.to_csv("metricas_modelos_classificacao_sem_nulos2.csv")
dataframe_metricas_modelos = dataframe_metricas_modelos[dataframe_metricas_modelos['accuracy']>0.60]

In [100]:
dataframe_metricas_modelos.shape

(187, 10)

### 50 melhores acurácia geral

In [101]:
dataframe_metricas_modelos.sort_values(by=['accuracy'],ascending=False).reset_index(drop=False).head(50)

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
0,92,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,0.029968,52.743143
1,90,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,2.650857,59.318689
2,88,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,0.028922,52.933302
3,86,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,2.657073,59.128510
4,85,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,2.737489,59.118438
5,91,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,0.035696,53.158206
6,89,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,2.536995,59.317578
7,87,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,0.036204,52.918117
8,12,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026929,1.939642
9,20,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,2.581278,59.302167


### 50 melhores f1-score geral

In [102]:
dataframe_metricas_modelos.sort_values(by=['media f1-score'],ascending=False).reset_index(drop=False).head(50)

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
0,92,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,0.029968,52.743143
1,90,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,2.650857,59.318689
2,88,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,0.028922,52.933302
3,86,KNeighborsClassifier,0.878771,0.876904,0.874532,0.875048,0.881079,0.878771,0.879332,2.657073,59.128510
4,85,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,2.737489,59.118438
5,91,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,0.035696,53.158206
6,89,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,2.536995,59.317578
7,87,KNeighborsClassifier,0.870861,0.871357,0.865837,0.867745,0.873307,0.870861,0.871305,0.036204,52.918117
8,14,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,2.654883,59.306390
9,24,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026941,2.180462


### 10 piores acurácia geral

In [119]:
dataframe_metricas_modelos.dropna().sort_values(by=['accuracy'],ascending=False).reset_index(drop=False)[60:120]

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
60,9,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,3.602799,81.478564
61,7,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,2.621779,59.485099
62,21,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,3.614048,82.147169
63,15,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,3.536194,81.705079
64,19,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,2.674072,59.336886
65,17,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,0.027099,2.162479
66,79,KNeighborsClassifier,0.853017,0.854480,0.845563,0.849229,0.853952,0.853017,0.852657,0.035496,100.371877
67,81,KNeighborsClassifier,0.853017,0.854480,0.845563,0.849229,0.853952,0.853017,0.852657,2.524588,108.853681
68,83,KNeighborsClassifier,0.853017,0.854480,0.845563,0.849229,0.853952,0.853017,0.852657,0.037769,100.132839
69,77,KNeighborsClassifier,0.853017,0.854480,0.845563,0.849229,0.853952,0.853017,0.852657,2.582238,108.852259


### 10 piores fi-score geral

In [104]:
dataframe_metricas_modelos.sort_values(by=['media f1-score'],ascending=False).reset_index(drop=False).tail(10)

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
177,2,MultinomialNB,0.655813,0.704017,0.682515,0.657349,0.726818,0.655813,0.651434,0.011921,0.000000
178,4,MultinomialNB,0.655813,0.704017,0.682515,0.657349,0.726818,0.655813,0.651434,0.037762,0.009975
179,6,DecisionTreeClassifier,0.662436,0.656639,0.657211,0.656877,0.663267,0.662436,0.662805,0.258929,0.015926
180,3,DecisionTreeClassifier,0.657469,0.650561,0.651001,0.650693,0.658803,0.657469,0.658051,0.171860,0.000000
181,14,MLPClassifier,0.645511,0.485123,0.600357,0.523310,0.495820,0.645511,0.546858,243.919468,0.044364
182,15,MLPClassifier,0.645511,0.485123,0.600357,0.523310,0.495820,0.645511,0.546858,245.062117,0.029921
183,13,MLPClassifier,0.645511,0.485123,0.600357,0.523310,0.495820,0.645511,0.546858,244.065341,0.031241
184,4,MLPClassifier,0.645511,0.485123,0.600357,0.523310,0.495820,0.645511,0.546858,243.787861,0.031245
185,5,MLPClassifier,0.645511,0.485123,0.600357,0.523310,0.495820,0.645511,0.546858,242.884969,0.031243
186,6,MLPClassifier,0.645511,0.485123,0.600357,0.523310,0.495820,0.645511,0.546858,243.016130,0.036901


### 50 melhores tempos de fit geral

In [105]:
dataframe_metricas_modelos.sort_values(by=['tempo fit'],ascending=True).reset_index(drop=False).head(50)

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
0,2,MultinomialNB,0.655813,0.704017,0.682515,0.657349,0.726818,0.655813,0.651434,0.011921,0.000000
1,48,KNeighborsClassifier,0.803716,0.800377,0.797787,0.798698,0.805042,0.803716,0.804033,0.025927,50.855635
2,42,KNeighborsClassifier,0.803716,0.800377,0.797787,0.798698,0.805042,0.803716,0.804033,0.026132,50.922834
3,36,KNeighborsClassifier,0.865526,0.860896,0.861536,0.861201,0.865854,0.865526,0.865675,0.026929,50.943763
4,12,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026929,1.939642
5,24,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026941,2.180462
6,18,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026942,2.032678
7,70,KNeighborsClassifier,0.865526,0.860896,0.861536,0.861201,0.865854,0.865526,0.865675,0.026961,51.084528
8,17,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,0.027099,2.162479
9,23,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,0.027233,2.006593


### 50 melhores tempos de predict geral

In [106]:
dataframe_metricas_modelos.sort_values(by=["tempo predict"],ascending=True).reset_index(drop=False).head(50)

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
0,30,LogisticRegression,0.839036,0.839229,0.838130,0.837228,0.844115,0.839036,0.840270,82.306628,0.000000
1,44,LogisticRegression,0.841244,0.842240,0.842142,0.839821,0.848847,0.841244,0.842855,19.012440,0.000000
2,3,DecisionTreeClassifier,0.657469,0.650561,0.651001,0.650693,0.658803,0.657469,0.658051,0.171860,0.000000
3,8,SGDClassifier,0.811258,0.806286,0.807612,0.806837,0.812522,0.811258,0.811778,54.311285,0.000000
4,2,MultinomialNB,0.655813,0.704017,0.682515,0.657349,0.726818,0.655813,0.651434,0.011921,0.000000
5,2,MLPClassifier,0.863135,0.860762,0.857728,0.858958,0.862484,0.863135,0.862535,141.878605,0.008004
6,8,LogisticRegression,0.841060,0.842159,0.840436,0.839503,0.847241,0.841060,0.842535,5.576324,0.008976
7,5,LogisticRegression,0.839036,0.839229,0.838130,0.837228,0.844115,0.839036,0.840270,38.598322,0.008978
8,43,LogisticRegression,0.841611,0.842675,0.842552,0.840258,0.849185,0.841611,0.843222,6.422049,0.008978
9,7,SGDClassifier,0.804084,0.799338,0.798517,0.798913,0.803798,0.804084,0.803925,58.237993,0.008979


### 50 melhores tempos de fit para acurácia acima de 82%

In [111]:
dataframe_metricas_modelos[dataframe_metricas_modelos['accuracy']>0.82].sort_values(by=["tempo fit"],ascending=True).reset_index(drop=False).head(50)

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
0,36,KNeighborsClassifier,0.865526,0.860896,0.861536,0.861201,0.865854,0.865526,0.865675,0.026929,50.943763
1,12,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026929,1.939642
2,24,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026941,2.180462
3,18,KNeighborsClassifier,0.865710,0.861384,0.861755,0.861432,0.866630,0.865710,0.866047,0.026942,2.032678
4,70,KNeighborsClassifier,0.865526,0.860896,0.861536,0.861201,0.865854,0.865526,0.865675,0.026961,51.084528
5,17,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,0.027099,2.162479
6,23,KNeighborsClassifier,0.856880,0.854537,0.851830,0.852997,0.857339,0.856880,0.856932,0.027233,2.006593
7,30,KNeighborsClassifier,0.865526,0.860896,0.861536,0.861201,0.865854,0.865526,0.865675,0.027902,50.809997
8,76,KNeighborsClassifier,0.865526,0.860896,0.861536,0.861201,0.865854,0.865526,0.865675,0.027922,51.050041
9,84,KNeighborsClassifier,0.861295,0.858519,0.855271,0.856723,0.861420,0.861295,0.861185,0.027959,99.976362


### 50 melhores tempos de predict para acurácia acima de 82%

In [108]:
dataframe_metricas_modelos[dataframe_metricas_modelos['accuracy']>0.82].sort_values(by=["tempo predict"],ascending=True).reset_index(drop=False).head(50)

,index,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
0,44,LogisticRegression,0.841244,0.842240,0.842142,0.839821,0.848847,0.841244,0.842855,19.012440,0.000000
1,30,LogisticRegression,0.839036,0.839229,0.838130,0.837228,0.844115,0.839036,0.840270,82.306628,0.000000
2,2,MLPClassifier,0.863135,0.860762,0.857728,0.858958,0.862484,0.863135,0.862535,141.878605,0.008004
3,8,LogisticRegression,0.841060,0.842159,0.840436,0.839503,0.847241,0.841060,0.842535,5.576324,0.008976
4,5,LogisticRegression,0.839036,0.839229,0.838130,0.837228,0.844115,0.839036,0.840270,38.598322,0.008978
5,43,LogisticRegression,0.841611,0.842675,0.842552,0.840258,0.849185,0.841611,0.843222,6.422049,0.008978
6,10,MLPClassifier,0.863135,0.860762,0.857728,0.858958,0.862484,0.863135,0.862535,141.549502,0.009053
7,4,LogisticRegression,0.839036,0.839229,0.838130,0.837228,0.844115,0.839036,0.840270,17.214075,0.009957
8,22,LogisticRegression,0.840140,0.840395,0.839368,0.838427,0.845295,0.840140,0.841408,9.799134,0.009973
9,23,LogisticRegression,0.840324,0.841386,0.839653,0.838769,0.846408,0.840324,0.841791,23.224741,0.009976


In [120]:
dataframe_metricas_modelos.groupby("model").mean().sort_values(by=['accuracy'],ascending=False).reset_index(drop=False)

,model,accuracy,media precision,media recall,media f1-score,media ponderada precision,media ponderada recall,media ponderada f1-score,tempo fit,tempo predict
0,KNeighborsClassifier,0.861172,0.857953,0.856547,0.857146,0.861436,0.861172,0.861201,2.359133,61.661797
1,KNeighborsClassifier,0.851669,0.849318,0.846425,0.847539,0.852470,0.851669,0.851748,1.864382,57.349736
2,LogisticRegression,0.840140,0.840637,0.839370,0.838445,0.845588,0.840140,0.841460,18.495453,0.011134
3,LogisticRegression,0.840123,0.840632,0.839628,0.838457,0.845861,0.840123,0.841476,22.637263,0.011937
4,RandomForestClassifier,0.831494,0.832221,0.834532,0.829712,0.842888,0.831494,0.833584,27.562878,0.187485
5,ExtraTreesClassifier,0.828964,0.827474,0.830388,0.826334,0.838109,0.828964,0.830954,8.286375,0.200713
6,MLPClassifier,0.821008,0.799391,0.813333,0.803305,0.804524,0.821008,0.809220,349.548159,0.027282
7,GradientBoostingClassifier,0.818801,0.822087,0.823021,0.818307,0.831095,0.818801,0.820834,358.823953,0.067276
8,SGDClassifier,0.808453,0.814904,0.802872,0.802212,0.816551,0.808453,0.806075,21.767559,0.018263
9,SVC,0.797339,0.797991,0.789177,0.789754,0.800500,0.797339,0.795509,86.793537,11.631763
